<a href="https://colab.research.google.com/github/choi-yongsuk/deep-learning-nlp/blob/master/nlp-article.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

#sentence = ("오늘도 폭염이 이어졌는데요, 내일은 반가운 비 소식이 있습니다.",
#            "오늘도 폭염이 이어졌는데요, 내일은 반가운 비 소식이 있습니다.",
#            "폭염을 피해 놀러왔다가 갑작스런 비로 망연자실하고 있습니다.")
#            "123.")


sentence = ("오늘도 폭염이 이어졌는데요, 내일은 반가운 비 소식이 있습니다.",
            "바나나 바나나 바나나 바나나 바나나.",
#            "폭염을 피해 놀러왔다가 갑작스런 비로 망연자실하고 있습니다.")
            "바나나 바나나 .",
            "바나나 바나나 .",
            "바나나 바나나 .",            
            "바나나.")

tfidf_vectorizer = TfidfVectorizer(max_features=100)
tfidf_matrix = tfidf_vectorizer.fit_transform(sentence) # vectorization

from sklearn.metrics.pairwise import cosine_similarity
print("[1] and [2]", cosine_similarity(tfidf_matrix[0], tfidf_matrix[1]))
print("[1] and [3]", cosine_similarity(tfidf_matrix[0], tfidf_matrix[2]))

[1] and [2] [[0.]]
[1] and [3] [[0.]]


In [45]:
array = ["F", "D", "A", "C", "A", "C", "F", "B", "C", "E", "D", "C", "F", "A", "B", "E", "F", "E"]
my_dict = {'a': array, 'b': 2, 'c': 3, 'd': 2, 'e': 1}
print(type(my_dict))
print(my_dict['a'])

<class 'dict'>
['F', 'D', 'A', 'C', 'A', 'C', 'F', 'B', 'C', 'E', 'D', 'C', 'F', 'A', 'B', 'E', 'F', 'E']


중복제거 코드

In [21]:
from functools import reduce

array = ["F", "D", "A", "C", "A", "C", "F", "B", "C", "E", "D", "C", "F", "A", "B", "E", "F", "E"]

result = reduce(lambda acc, cur: acc if cur in acc else acc+[cur], array, [])

print("중복 제거 전 : ", array)
print("중복 제거 후 : ", result)


중복 제거 전 :  ['F', 'D', 'A', 'C', 'A', 'C', 'F', 'B', 'C', 'E', 'D', 'C', 'F', 'A', 'B', 'E', 'F', 'E']
중복 제거 후 :  ['F', 'D', 'A', 'C', 'B', 'E']


In [25]:
my_dict = {'a': 1, 'b': 2, 'c': 3, 'd': 2, 'e': 1}

seen = []
result = dict()
for key, val in my_dict.items():
    if val not in seen:
        seen.append(val)
        result[key] = val

print(f'Original dict: {my_dict}')
print(f'Dict after removal: {result}')

Original dict: {'a': 1, 'b': 2, 'c': 3, 'd': 2, 'e': 1}
Dict after removal: {'a': 1, 'b': 2, 'c': 3}


word embedding 실습

In [46]:
#!pip install feedparser
#!pip install newspaper3k
#!pip install konlpy

#!pip install bs4
#!sudo apt-get install -y fonts-nanum
#!sudo fc-cache -fv
#!rm ~/.cache/matplotlib -rf

import feedparser
from newspaper import Article
from konlpy.tag import Okt
from collections import Counter
from operator import eq
from functools import reduce
import numpy as np
from bs4 import BeautifulSoup

urls = ["http://rss.etnews.com/Section901.xml",
        "http://rss.etnews.com/Section902.xml",
        "http://rss.etnews.com/Section903.xml",
        "http://rss.etnews.com/Section904.xml"]

#[1]
def crawl_rss(urls):
  arr_rss = []
  for url in urls:
#->    print("[Crawl RSS]", url)
    parse_rss = feedparser.parse(url);
    for p in parse_rss.entries:
        arr_rss.append({'title':p.title, 'link':p.link})
  return arr_rss

#[3-1] url에서 기사를 읽어 온다.
def crawl_article(url, language='ko'):
#-> print("[Crawl Article]",url)
  var_article = Article(url, language=language)
  var_article.download()
  var_article.parse()
  return var_article.title, var_article.text

#[4-1] 읽어온 기사에서 키워드의 빈도를 측정한다.
def get_keywords(text, nKeywords=50):
  spliter = Okt()
  nouns = spliter.nouns(text)
  count = Counter(nouns)
  return_list = []
  for n, c in count.most_common(nKeywords):
    temp = {'keyword':n, 'count': c}
    return_list.append(temp)
    #print(temp)
  return return_list

#[5-1] 입력된 단어를 키워드 리스트에서 검색하고 결과를 돌려 준다.  
def Howmanywords(request, keyword):
  nWords = 0
  for n in keyword:
    noun = n['keyword']
    count = n['count']
    if eq(noun, request):
      nWords = count
  return nWords

#[2]
article_list = crawl_rss(urls)
print(article_list)
#print(*article_list,sep='\n')     #{title: ????, link: ????}

'''
#[3]
# Part 2
for article in article_list:
  _, text = crawl_article(article['link'])
  article['text'] = text
#  print(text)


#[4]
# Part 3 : 수집된 텍스트 에서 키워드, 빈도를 리스트로 만든다.
print('[Parsing Text]')
for article in article_list:
  keywords = get_keywords(article['text'])
  
  print(f'keywords: {keywords.type()}')
  article['keywords'] = keywords

print(f'article: {article}')
print(f'Original dict:', len(article.keys()))
'''

'''
seen = []
result = dict()
for key, val in article.items():
    if val not in seen:
        seen.append(val)
        result[key] = val

print(f'Dict after removal:', len(result.keys()))
'''

'''
#[5]
# Part 4: 입력된 키워드 검색
print('Enter your query: ')
query = input()
for result in article_list:
  n = Howmanywords(query, result['keywords'])
  if n!=0:
    print('[TF]:', n, '[Title]:', result['title'], '[URL]:', result['link'],'\n')
'''

#[5]
# Part 4: 입력된 키워드 검색
#print('Enter your query: ')
#query = input()
#for article in article_list:
#  n = Howmanywords(query, article['keywords'])
#  if n!=0:
#    print('[TF]:', n, '[Title]:', article['title'], '[URL]:', article['link'],'\n')


AttributeError: ignored